# DoubleDrift

#### Project description:
_Adaptation of experiment 1 (perception) of Lisa & Cavanagh, 2015, Current Biology<br>
(http://dx.doi.org/10.1016/j.cub.2015.08.021) for the AMU Neuroscience Master APP 2024 courses._

#### Hypothesis: 
_Participants mislocalize perceptively the drifting gabor but saccade to correctly to its<br>
 physical position_
 
#### Exercice for APP2024:
_Analyse data to reach an adapted version of Figure 1 of [Lisa & Cavanagh, 2015, Current Biology](http://dx.doi.org/10.1016/j.cub.2015.08.021)_

<img src="img/Lisi_Cavanagh_2015_CB_Figure1.png" width=700 alt="Figure 1">
<!-- ![Lisi_Cavanagh_2015_CB_Figure1.png]()  -->

#### Perception data analysis:
- [x] psychometric staircase
- [x] psychometric function

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import psychofit as psy

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plot_utils import plotly_template
import plotly.express as px

In [ ]:
# Define folders
base_dir = '..'
data_dir = '{}/data'.format(base_dir)
subject = 'sub-01'
session = 'ses-01'
subject_num = subject[4:]
fig_dir = '{}/{}/{}/beh/figures'.format(data_dir, subject, session)

In [ ]:
# Load dataframe
df_all = pd.read_csv('{}/{}/{}/beh/{}_task-DoubleDriftPerception_data.csv'.format(data_dir, subject, session, subject))
df_pf_value = pd.read_csv('{}/{}/{}/beh/{}_task-DoubleDriftPerception_pfvalue.csv'.format(data_dir, subject, session, subject))

#### Perception analysis output graph

In [ ]:
# general figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Helvetica',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

# figure specific settings
colors_panel = px.colors.qualitative.Prism
subplot_titles = ('<b>Psychometric staircase run-01<b>', '<b>Psychometric staircase run-02<b>')

runs = ['run-01', 'run-02']
scs = ['1st staircase', '2nd staircase']
sc_cols = [colors_panel[1], colors_panel[7]]
x_legend = 1.1
y_legend_step = 0.08
y_legend_sc = 0.8
rad_leg = 0.04

# subplot settings
rows, cols = 1, 2
fig_height, fig_width = 300, 500 * cols +100
column_widths, row_heights = [1,1],[1]
sb_specs = [[{},{}]]
fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.2, horizontal_spacing=0.15, 
                    column_widths=column_widths, row_heights=row_heights, subplot_titles=subplot_titles)
# Right / Left saccades
# ---------------------
for run_num, run in enumerate(runs):
    for sc_num, sc in enumerate(scs):
        if sc_num == 0:y_txt = [-65]
        elif sc_num == 1:y_txt = [-75]
        # plot
        df_cond = df_all.loc[(df_all.staircase_num == sc_num+1) & (df_all.run_number==run_num+1)]
        
        fig.append_trace(go.Scatter(x=df_cond.trial_number,
                                    y=-df_cond.ext_mot_ori,
                                    name=sc, mode='markers+lines', marker_color=sc_cols[sc_num], 
                                    marker_size=8, marker_opacity=1, line_width=2,
                                    marker_line_width=0, marker_symbol='circle'), row=1, col=run_num+1)

        # compute pse
        pse_cond = -df_cond.ext_mot_ori[-26:-1].mean()
        x_pse = np.arange(0,101,1)
        y_pse = x_pse * 0 + pse_cond
        fig.append_trace(go.Scatter(x=x_pse, y=y_pse, mode='lines', marker_color=sc_cols[sc_num], 
                                    line_dash='dot', line_width=2), row=1, col=run_num+1)
        fig.append_trace(go.Scatter(x=[85], y=y_txt, mode="text", text="μ = {:2.1f} deg".format(pse_cond),
                         textfont_color=sc_cols[sc_num], textfont_size=15), row=1, col=run_num+1)
        
        # legend
        fig.add_annotation(xref='paper', yref='paper', x=x_legend, y=y_legend_sc - y_legend_step * sc_num, 
                           text=sc, showarrow=False, font_size = template_specs['axes_font_size'], xanchor = 'left', yanchor='middle')
        fig.add_shape(type='circle', xref='paper', yref='paper', fillcolor=sc_cols[sc_num],line_width=0,
                      x0=x_legend - 0.02 - rad_leg / 9, y0=y_legend_sc - y_legend_step * sc_num - rad_leg / 2, 
                      x1=x_legend - 0.02 + rad_leg / 9, y1=y_legend_sc - y_legend_step * sc_num + rad_leg / 2)
    
# axes settings
for row in np.arange(rows):
    for col in np.arange(cols):
            
        fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], 
                         range=[0,100], title='Trial number', row=row+1, col=col+1)
        fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], 
                         title='Path orientation (deg)', range=[-80,0], row=row+1, col=col+1)
    
# Save figure
fig.layout.update(template=fig_template, width=fig_width, height=fig_height, margin_l=50, 
                  margin_r=200, margin_t=50, margin_b=50,  showlegend=False)
fig.show()
if not os.path.exists(fig_dir):os.mkdir(fig_dir)
fig.write_image('{}/{}_staircase.pdf'.format(fig_dir, subject))

In [ ]:
# subplot settings
rows, cols = 1, 1
fig_height, fig_width = 300, 400 * cols + 100
column_widths, row_heights = [1],[1]
sb_specs = [[{}]]
subplot_titles = ('<b>Psychometric function<b>', '')
fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.2, horizontal_spacing=0.15, 
                    column_widths=column_widths, row_heights=row_heights, subplot_titles=subplot_titles)
df_pf_value['report_right_num_norm'] = np.array((df_pf_value.report_right_num-np.min(df_pf_value.report_right_num))/
                                                (np.max(df_pf_value.report_right_num)-np.min(df_pf_value.report_right_num)))


# compute psychometric function
# Parameter bounds are in the form [bias, threshold, lapse]
params = {'parmin': np.array([-100., 0., 0.]), # The minimum allowable parameter values
          'parmax': np.array([0., 40., 0.2]), # The maximum allowable parameter values
          'parstart': np.array([-40, 10., 0.1]), # Non-zero starting parameters, used to try to avoid local minima
          'nfits': 30} # The number of fits to run

data = np.vstack((np.array(df_pf_value.angle_test),
                  np.array(df_pf_value.report_right_num),
                  np.array(df_pf_value.report_right_prct)
                  ))
pars, L = psy.mle_fit_psycho(data, 'erf_psycho', **params)
fit_x = np.linspace(-80,0,50)
fit_y = psy.erf_psycho(pars, fit_x)
fig.append_trace(go.Scatter(x=fit_x, y=fit_y,
                            mode='lines', marker_color=colors_panel[7], marker_opacity=1, 
                            line_width=3, marker_symbol='circle'), row=1, col=1)
fig.append_trace(go.Scatter(x=[-20], y=[0.15], mode="text", text="μ = {:2.1f} deg".format(pars[0]),
                         textfont_color=colors_panel[7], textfont_size=15), row=1, col=1)

# plot dots
fig.append_trace(go.Scatter(x=df_pf_value.angle_test,
                            y=df_pf_value.report_right_prct,
                            marker_size=df_pf_value.report_right_num_norm*20, 
                            mode='markers', marker_color=colors_panel[7], marker_opacity=0.5, line_width=3,
                            marker_line_width=0, marker_symbol='circle'), row=1, col=1)
fig.add_hline(y=0.5, opacity=0.25, row=1, col=1)

# axes settings
for row in np.arange(rows):
    for col in np.arange(cols):
        fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], 
                         range=[-80, 0], title='Trial number', row=row+1, col=col+1)
        fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], 
                         title='Report right (%)', range=[0,1], row=row+1, col=col+1)
fig.layout.update(template=fig_template, width=fig_width, height=fig_height, margin_l=50, 
                  margin_r=200, margin_t=50, margin_b=50,  showlegend=False)
fig.show()
if not os.path.exists(fig_dir):os.mkdir(fig_dir)
fig.write_image('{}/{}_pfun.pdf'.format(fig_dir, subject))